<a href="https://colab.research.google.com/github/bryanbayup/Machine-Learning/blob/main/finalisasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Persiapan Linkungan



In [1]:
# Install library yang diperlukan
!pip install gensim
!pip install tensorflow keras-tuner
!pip install google-api-python-client
!pip install seqeval
!pip install imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=7308dfaa834e5879d3a1985f8324641ee9bf38d321d2190563ec7b098890cb80
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
# Import library
import json
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import gensim
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from seqeval.metrics import classification_report as seq_classification_report
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.regularizers import l2
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from imblearn.over_sampling import RandomOverSampler
import pickle
import os
from google.colab import drive

<ipython-input-2-121ea54b41c5>:19: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


# 2.Mount Google Drive

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# 3.Download dan Persiapan FastText Embeddings

In [4]:
from gensim.models import KeyedVectors

# URL FastText Bahasa Indonesia
fasttext_url = 'https://www.dropbox.com/scl/fi/zjvzebgdklqosxpylu93b/id.tar.gz?rlkey=k8n322p8xteuog6lf3jdg2a6t&st=8wwkv788&dl=1'

# Path untuk menyimpan di Google Drive
drive_path = '/content/drive/MyDrive/dataset'
os.makedirs(drive_path, exist_ok=True)

# Unduh file dan simpan ke Google Drive
file_name = 'id.tar.gz'
drive_file_path = os.path.join(drive_path, file_name)
# !wget -O "{drive_file_path}" "{fasttext_url}"
# print(f"File telah diunduh dan disimpan di: {drive_file_path}")

# Ekstrak file ke direktori aktif saat ini
!tar -xzf "{drive_file_path}" -C .
print(f"File telah diekstrak ke direktori aktif: {os.getcwd()}")

# Muat model FastText dari file yang diekstrak
try:
    fasttext_model = KeyedVectors.load_word2vec_format('id.vec', binary=False)
    print("Model FastText 'id.vec' berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat 'id.vec': {e}")
    # Jika gagal, coba muat 'id.bin'
    try:
        fasttext_model = KeyedVectors.load_facebook_vectors('id.bin')
        print("Model FastText 'id.bin' berhasil dimuat.")
    except Exception as e:
        print(f"Gagal memuat 'id.bin': {e}")
        raise ValueError("Gagal memuat model FastText. Pastikan file 'id.vec' atau 'id.bin' dalam format yang benar.")

File telah diekstrak ke direktori aktif: /content
Model FastText 'id.vec' berhasil dimuat.


# 4.Preprocessing Data

In [5]:
# Memuat dataset dari file JSON
with open('/content/drive/MyDrive/dataset/dataaa.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Mengubah dataset menjadi DataFrame
df = pd.DataFrame(data)

# Tampilakan 5 dataset paling atas
df.head()

,utterances,intent,entities,responses
0,Anjing saya mengalami gatal-gatal terus meneru...,medical_inquiry_dog,"[{'entity': 'animal', 'value': 'Anjing', 'star...","Gunakan sampo hipoalergenik, oleskan salep hid..."
1,Anjing saya terlihat sering muntah dan kehilan...,medical_inquiry_dog,"[{'entity': 'animal', 'value': 'Anjing', 'star...",Berikan cairan elektrolit untuk mencegah dehid...
2,"Anjing saya terlihat lesu, demam, dan tidak ma...",medical_inquiry_dog,"[{'entity': 'animal', 'value': 'Anjing', 'star...","Pastikan anjing tetap terhidrasi, gunakan komp..."
3,Anjing saya mengalami batuk kering dan nafasny...,medical_inquiry_dog,"[{'entity': 'animal', 'value': 'Anjing', 'star...",Berikan obat batuk khusus anjing yang disarank...
4,Anjing saya mengalami luka pada kulit yang men...,medical_inquiry_dog,"[{'entity': 'animal', 'value': 'Anjing', 'star...","Bersihkan luka dengan antiseptik, oleskan sale..."


In [6]:
# Menentukan jumlah sampel maksimum per intent
max_samples = 50

df_list = []
for intent in df['intent'].unique():
    df_intent = df[df['intent'] == intent]
    if len(df_intent) > max_samples:
        df_intent = resample(df_intent, replace=False, n_samples=max_samples, random_state=42)
    df_list.append(df_intent)

df_balanced = pd.concat(df_list).reset_index(drop=True)

# Encode intents
label_encoder = LabelEncoder()
df_balanced['intent_label'] = label_encoder.fit_transform(df_balanced['intent'])

# Simpan mapping label
intent_mapping = dict(zip(df_balanced['intent_label'], df_balanced['intent']))

# Mengatasi kelas imbalanced dengan oversampling
ros = RandomOverSampler(random_state=42)

X_ros, y_ros = ros.fit_resample(df_balanced[['utterances', 'responses']], df_balanced['intent_label'])

df_balanced = pd.DataFrame({
    'utterances': X_ros['utterances'],
    'responses': X_ros['responses'],
    'intent_label': y_ros
})

# Filter data yang memiliki entitas
df_ner = df[df['entities'].map(lambda d: len(d)) > 0].reset_index(drop=True)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Pembersihan teks
df_balanced['utterances_clean'] = df_balanced['utterances'].apply(clean_text)
df_ner['utterances_clean'] = df_ner['utterances'].apply(clean_text)

texts = df_balanced['utterances_clean'].tolist()
labels = df_balanced['intent_label'].tolist()

# Split data untuk Klasifikasi Intent
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

# 5.Tokenisasi dan Pembuatan Embedding Matrix

In [7]:
# Tokenisasi
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Mengonversi teks ke sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

# Padding sequences
max_seq_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in val_sequences))
train_padded = pad_sequences(train_sequences, maxlen=max_seq_length, padding='post')
val_padded = pad_sequences(val_sequences, maxlen=max_seq_length, padding='post')

# Mengonversi labels ke categorical
num_classes = len(label_encoder.classes_)
train_labels_cat = to_categorical(train_labels, num_classes=num_classes)
val_labels_cat = to_categorical(val_labels, num_classes=num_classes)

# Membuat embedding matrix menggunakan FastText
embedding_dim = fasttext_model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in word_index.items():
    if word in fasttext_model:
        embedding_matrix[idx] = fasttext_model[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

# 6.Persiapan Data untuk NER

In [8]:
def prepare_ner_data(df, tokenizer, max_seq_length):
    texts = []
    labels = []
    for index, row in df.iterrows():
        text = row['utterances_clean']
        entities = row['entities']
        tokens = tokenizer.texts_to_sequences([text])[0]
        label_seq = ['O'] * len(tokens)
        for ent in entities:
            ent_text = clean_text(ent['value'])
            ent_tokens = tokenizer.texts_to_sequences([ent_text])[0]
            ent_len = len(ent_tokens)
            for i in range(len(tokens) - ent_len + 1):
                if tokens[i:i+ent_len] == ent_tokens:
                    label_seq[i] = 'B-' + ent['entity']
                    for j in range(1, ent_len):
                        label_seq[i+j] = 'I-' + ent['entity']
                    break
        texts.append(tokens)
        labels.append(label_seq)
    # Padding
    texts_padded = pad_sequences(texts, maxlen=max_seq_length, padding='post')
    # Padding labels
    labels_padded = [label + ['O']*(max_seq_length - len(label)) for label in labels]
    return texts_padded, labels_padded

# Membuat label encoder untuk NER
all_labels = set()
for label_list in df_ner['entities']:
    for ent in label_list:
        all_labels.add('B-' + ent['entity'])
        all_labels.add('I-' + ent['entity'])
all_labels.add('O')
ner_label_encoder = {label: idx for idx, label in enumerate(sorted(all_labels))}
ner_label_decoder = {idx: label for label, idx in ner_label_encoder.items()}

# Siapkan data NER
texts_ner, labels_ner = prepare_ner_data(df_ner, tokenizer, max_seq_length)

# Mengonversi labels ke format numerik dan categorical
def encode_ner_labels(labels, ner_label_encoder):
    labels_encoded = []
    for label_seq in labels:
        label_ids = [ner_label_encoder[label] for label in label_seq]
        labels_encoded.append(label_ids)
    labels_encoded = np.array(labels_encoded)
    labels_encoded = to_categorical(labels_encoded, num_classes=len(ner_label_encoder))
    return labels_encoded

labels_ner_encoded = encode_ner_labels(labels_ner, ner_label_encoder)

# Split data untuk NER
train_texts_ner, val_texts_ner, train_labels_ner, val_labels_ner = train_test_split(
    texts_ner,
    labels_ner_encoded,
    test_size=0.1,
    random_state=42,
)

# 7.Definisi dan Kompilasi Model

In [9]:
from tensorflow.keras.layers import Dense, Input, Dropout, Bidirectional, LSTM, TimeDistributed
from tensorflow.keras.models import Model

# Definisikan Fungsi untuk Membangun Model Klasifikasi Intent
def build_intent_model(embedding_matrix, max_seq_length, num_classes, l2_reg=0.001):
    inputs = Input(shape=(max_seq_length,))
    embedding = tf.keras.layers.Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=max_seq_length,
        trainable=False
    )(inputs)
    lstm = Bidirectional(LSTM(64, kernel_regularizer=l2(l2_reg), return_sequences=False))(embedding)
    dense = Dense(64, activation='relu', kernel_regularizer=l2(l2_reg))(lstm)
    dropout = Dropout(0.5)(dense)
    outputs = Dense(num_classes, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Definisikan Fungsi untuk Membangun Model NER
def build_ner_model(embedding_matrix, max_seq_length, num_entities, l2_reg=0.001):
    inputs = Input(shape=(max_seq_length,))
    embedding = tf.keras.layers.Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=max_seq_length,
        trainable=False
    )(inputs)
    lstm = Bidirectional(LSTM(64, kernel_regularizer=l2(l2_reg), return_sequences=True))(embedding)
    dropout = Dropout(0.5)(lstm)
    outputs = TimeDistributed(Dense(num_entities, activation='softmax', kernel_regularizer=l2(l2_reg)))(dropout)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Membangun Model Klasifikasi Intent
model_intent = build_intent_model(embedding_matrix, max_seq_length, num_classes, l2_reg=0.001)
model_intent.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_intent.summary()

# Membangun Model NER
model_ner = build_ner_model(embedding_matrix, max_seq_length, len(ner_label_encoder), l2_reg=0.001)
model_ner.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_ner.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 28)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 28, 300)             │         184,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │         186,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 15)                  │             975 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 380,311 (1.45 MB)

 Trainable params: 196,111 (766.06 KB)

...

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 28)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, 28, 300)             │         184,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 28, 128)             │         186,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 28, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 28, 17)              │           2,193 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 373,273 (1.42 MB)

 Trainable params: 189,073 (738.57 KB)

 Non-trainable params: 184,200 (719.53 KB)

# 8.Pelatihan Model dengan Hyperparameter Optimization

In [10]:
from kerastuner import HyperModel

# Definisikan HyperModel untuk Intent Classification
class IntentHyperModel(HyperModel):
    def __init__(self, embedding_matrix, max_seq_length, num_classes):
        self.embedding_matrix = embedding_matrix
        self.max_seq_length = max_seq_length
        self.num_classes = num_classes

    def build(self, hp):
        l2_reg = hp.Choice('l2_reg', values=[1e-4, 1e-3, 1e-2])
        dropout_rate = hp.Float('dropout_rate', 0.3, 0.7, step=0.1)
        lstm_units = hp.Int('lstm_units', min_value=32, max_value=128, step=32)
        dense_units = hp.Int('dense_units', min_value=32, max_value=128, step=32)

        inputs = Input(shape=(self.max_seq_length,))
        embedding = tf.keras.layers.Embedding(
            input_dim=self.embedding_matrix.shape[0],
            output_dim=self.embedding_matrix.shape[1],
            weights=[self.embedding_matrix],
            input_length=self.max_seq_length,
            trainable=False
        )(inputs)
        lstm = Bidirectional(LSTM(lstm_units, kernel_regularizer=l2(l2_reg), return_sequences=False))(embedding)
        dense = Dense(dense_units, activation='relu', kernel_regularizer=l2(l2_reg))(lstm)
        dropout = Dropout(dropout_rate)(dense)
        outputs = Dense(self.num_classes, activation='softmax')(dropout)
        model = Model(inputs=inputs, outputs=outputs)

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

# Inisialisasi HyperModel
intent_hypermodel = IntentHyperModel(embedding_matrix, max_seq_length, num_classes)

# Inisialisasi RandomSearch
tuner_intent = RandomSearch(
    intent_hypermodel,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='intent_tuner_dir',
    project_name='intent_classification'
)

# Menampilkan ringkasan tuner
tuner_intent.search_space_summary()

# Pencarian Hyperparameter untuk Intent Classification
tuner_intent.search(
    train_padded,
    train_labels_cat,
    epochs=10,
    validation_data=(val_padded, val_labels_cat),
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    ]
)

# Mendapatkan model terbaik untuk Intent Classification
best_model_intent = tuner_intent.get_best_models(num_models=1)[0]
best_hp_intent = tuner_intent.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters for Intent Classification: {best_hp_intent.values}")

Trial 10 Complete [00h 00m 31s]
val_accuracy: 0.9466666579246521

Best val_accuracy So Far: 0.9633333086967468
Total elapsed time: 00h 07m 51s
Best Hyperparameters for Intent Classification: {'l2_reg': 0.0001, 'dropout_rate': 0.3, 'lstm_units': 32, 'dense_units': 96}


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# 9.Pelatihan Model NER

In [11]:
# Mempersiapkan Callback untuk NER
callbacks_ner = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint(
        filepath='best_model_ner.keras',
        save_best_only=True,
        save_weights_only=False
    )
]

# Melatih Model NER
history_ner = model_ner.fit(
    train_texts_ner,
    train_labels_ner,
    validation_data=(val_texts_ner, val_labels_ner),
    epochs=20,
    batch_size=16,
    callbacks=callbacks_ner
)

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - accuracy: 0.6189 - loss: 2.7636 - val_accuracy: 0.8023 - val_loss: 1.1408
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.8227 - loss: 1.0460 - val_accuracy: 0.8667 - val_loss: 0.7685
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.8694 - loss: 0.7564 - val_accuracy: 0.8841 - val_loss: 0.6034
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.8953 - loss: 0.5883 - val_accuracy: 0.9042 - val_loss: 0.5072
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.8999 - loss: 0.5033 - val_accuracy: 0.8990 - val_loss: 0.4579
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9099 - loss: 0.4452 - val_accuracy: 0.9111 - val_loss: 0.4080
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9144 - loss: 0.4033 - val_accuracy: 0.9077 - val_loss: 0.3835
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9207 - loss: 0.3594 - val_accuracy: 0.9120 - v

In [12]:
# Evaluasi Klasifikasi Intent
loss_intent, accuracy_intent = best_model_intent.evaluate(val_padded, val_labels_cat)
print(f'Akurasi Model Klasifikasi Intent: {accuracy_intent * 100:.2f}%')

# Prediksi pada data validasi Intent
val_preds_intent = best_model_intent.predict(val_padded)
val_preds_intent = np.argmax(val_preds_intent, axis=1)
val_true_intent = np.argmax(val_labels_cat, axis=1)

# Classification report untuk Intent
print("Classification Report untuk Intent:")
print(classification_report(val_true_intent, val_preds_intent, target_names=label_encoder.classes_))

# Evaluasi NER
loss_ner, accuracy_ner = model_ner.evaluate(val_texts_ner, val_labels_ner)
print(f'Akurasi Model NER: {accuracy_ner * 100:.2f}%')

# Prediksi pada data validasi NER
val_preds_ner = model_ner.predict(val_texts_ner)
val_preds_ner = np.argmax(val_preds_ner, axis=-1)
val_true_ner = np.argmax(val_labels_ner, axis=-1)

# Konversi label ke format aslinya
true_labels = []
pred_labels = []

for i in range(len(val_preds_ner)):
    true_label = []
    pred_label = []
    for j in range(len(val_preds_ner[i])):
        true_l = ner_label_decoder[val_true_ner[i][j]]
        pred_l = ner_label_decoder[val_preds_ner[i][j]]
        if true_l != 'O':
            true_label.append(true_l)
            pred_label.append(pred_l)
    true_labels.append(true_label)
    pred_labels.append(pred_label)

# Classification report untuk NER
print("Classification Report untuk NER:")
print(seq_classification_report(true_labels, pred_labels))

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9581 - loss: 0.3397  
Akurasi Model Klasifikasi Intent: 96.67%
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step
Classification Report untuk Intent:
                         precision    recall  f1-score   support

         cat_healthcare       1.00      1.00      1.00        10
 disease_prevention_cat       1.00      1.00      1.00        10
 disease_prevention_dog       1.00      1.00      1.00        10
         dog_healthcare       1.00      0.90      0.95        10
               end_chat       1.00      1.00      1.00        10
             intro_chat       1.00      1.00      1.00        10
 medical_inquiry_anjing       1.00      0.90      0.95        10
    medical_inquiry_cat       0.83      1.00      0.91        10
    medical_inquiry_dog       0.83      1.00      0.91        10
 medical_inquiry_kucing       1.00      0.90      0.95        10
    pet_food_suggestion       0.91      1.00      0.95        10
pet_health_status_check       1

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Membuat DataFrame utterances dan responses
df_utterances = df_balanced[['utterances', 'responses']].reset_index(drop=True)
df_utterances['utterances_clean'] = df_utterances['utterances'].apply(clean_text)

# Menghitung TF-IDF matrix
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_utterances['utterances_clean'])

def predict_intent(text):
    text_clean = clean_text(text)
    seq = tokenizer.texts_to_sequences([text_clean])
    padded_seq = pad_sequences(seq, maxlen=max_seq_length, padding='post')
    pred = best_model_intent.predict(padded_seq)
    predicted_label = np.argmax(pred, axis=1)[0]
    intent = label_encoder.inverse_transform([predicted_label])[0]
    return intent

def predict_entities(text):
    text_clean = clean_text(text)
    seq = tokenizer.texts_to_sequences([text_clean])
    padded_seq = pad_sequences(seq, maxlen=max_seq_length, padding='post')
    pred = model_ner.predict(padded_seq)
    pred_labels = np.argmax(pred, axis=-1)[0]
    tokens = tokenizer.sequences_to_texts(seq)[0].split()
    entities = []
    for idx, label_id in enumerate(pred_labels[:len(tokens)]):
        label = ner_label_decoder[label_id]
        if label != 'O':
            entities.append({'entity': label.split('-')[1], 'value': tokens[idx]})
    return entities

def get_response(user_input):
    # Preprocess input
    user_input_clean = clean_text(user_input)
    user_tfidf = vectorizer.transform([user_input_clean])

    # Hitung cosine similarity
    similarities = cosine_similarity(user_tfidf, tfidf_matrix)

    # Dapatkan indeks dengan similarity tertinggi
    most_similar_idx = np.argmax(similarities[0])

    # Jika similarity rendah, berikan respon default
    if similarities[0][most_similar_idx] < 0.35:
        return "Maaf, saya tidak memahami pertanyaan Anda."

    # Ambil respon yang sesuai
    response = df_utterances.iloc[most_similar_idx]['responses']

    return response

def chatbot_response(user_input):
    # Prediksi intent dan entitas
    intent = predict_intent(user_input)
    entities = predict_entities(user_input)

    response = get_response(user_input)

    return response

In [22]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Membuat widget input dan output
input_box = widgets.Text(
    value='',
    placeholder='Ketik pesan Anda...',
    description='Anda:',
    disabled=False
)

output_area = widgets.Output()

def on_submit(sender):
    user_input = input_box.value
    input_box.value = ''
    response = chatbot_response(user_input)
    with output_area:
        clear_output()
        print(f"Anda: {user_input}")
        print(f"Chatbot: {response}\n")

input_box.on_submit(on_submit)

display(input_box, output_area)

Text(value='', description='Anda:', placeholder='Ketik pesan Anda...')

Output()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


In [24]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (95.8 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 123629 files and directories currently instal

In [28]:
# Simpan model
best_model_intent.save('model_intent.keras')
model_ner.save('model_ner.keras')

# Simpan tokenizer dan label encoders
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('ner_label_encoder.pickle', 'wb') as handle:
    pickle.dump(ner_label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
!zip -r chatbot.zip model_intent.keras model_ner.keras tokenizer.pickle label_encoder.pickle ner_label_encoder.pickle vectorizer.pickle tfidf_matrix.pickle

  adding: model_intent.keras (deflated 10%)
  adding: model_ner.keras (deflated 8%)
  adding: tokenizer.pickle (deflated 46%)
  adding: label_encoder.pickle (deflated 38%)
  adding: ner_label_encoder.pickle (deflated 36%)
  adding: vectorizer.pickle (deflated 54%)
  adding: tfidf_matrix.pickle (deflated 47%)


In [30]:
# Simpan vectorizer
with open('vectorizer.pickle', 'wb') as handle:
    pickle.dump(vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Simpan TF-IDF matrix
with open('tfidf_matrix.pickle', 'wb') as handle:
    pickle.dump(tfidf_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)